In [17]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv("../../data/df_rfm.csv")

# Display the first few rows of the data
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
0,489434,85048,15CM_CHRISTMAS_GLASS_BALL_20_LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United_Kingdom,83.40
1,489434,22041,"RECORD_FRAME_7""_SINGLE_SIZE",48,2009-12-01 07:45:00,2.10,13085,United_Kingdom,100.80
2,489434,21232,STRAWBERRY_CERAMIC_TRINKET_BOX,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
3,489434,22064,PINK_DOUGHNUT_TRINKET_POT,24,2009-12-01 07:45:00,1.65,13085,United_Kingdom,39.60
4,489434,21871,SAVE_THE_PLANET_MUG,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
713597,581587,22613,PACK_OF_20_SPACEBOY_NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
713598,581587,22899,CHILDREN'S_APRON_DOLLY_GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
713599,581587,23254,CHILDRENS_CUTLERY_DOLLY_GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
713600,581587,23255,CHILDRENS_CUTLERY_CIRCUS_PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [19]:
# Compter le nombre de pays uniques pour chaque Customer_ID
unique_countries = df.groupby('Customer_ID')['Country'].nunique()

# Afficher les Customer_ID qui ont plus d'un pays
unique_countries[unique_countries > 1]

Customer_ID
12370    2
12394    2
12413    2
12417    2
12422    2
12423    2
12429    2
12431    2
12449    2
12455    2
12652    2
Name: Country, dtype: int64

In [20]:
unique_countries.value_counts()

1    5801
2      11
Name: Country, dtype: int64

In [21]:
df[df['Customer_ID'] == 12346]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
62205,499763,20682,RED_SPOTTY_CHILDS_UMBRELLA,1,2010-03-02 13:08:00,3.25,12346,United_Kingdom,3.25
62206,499763,20679,EDWARDIAN_PARASOL_RED,1,2010-03-02 13:08:00,5.95,12346,United_Kingdom,5.95
159681,513774,21524,DOORMAT_SPOTTY_HOME_SWEET_HOME,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159682,513774,22692,DOORMAT_WELCOME_TO_OUR_HOME,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159683,513774,22660,DOORMAT_I_LOVE_LONDON,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159684,513774,22687,DOORMAT_CHRISTMAS_VILLAGE,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159685,513774,22691,DOORMAT_WELCOME_SUNRISE,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159686,513774,48111,DOORMAT_3_SMILEY_CATS,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159687,513774,22690,DOORMAT_HOME_SWEET_HOME_BLUE,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49
159688,513774,21523,DOORMAT_FANCY_FONT_HOME_SWEET_HOME,1,2010-06-28 13:53:00,7.49,12346,United_Kingdom,7.49


In [22]:
from datetime import timedelta

# Convertir InvoiceDate de l'objet au format datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Obtenir la date maximale
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)

# Agréger les données (RFM : Récence, Fréquence, Valeur monétaire)
rfm = df.groupby(['Customer_ID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Calcul de la récence
    'Invoice': 'nunique',  # Comptage du nombre de transactions (fréquence)
    'TotalSales': 'sum'  # Somme des ventes (valeur monétaire)
})

# Renommer les colonnes
rfm.rename(columns={'InvoiceDate': 'Recency',
                   'Invoice': 'Frequency',
                   'TotalSales': 'MonetaryValue'}, inplace=True)

# Afficher les premières lignes des données
rfm



,Recency,Frequency,MonetaryValue
Customer_ID,,,
12346,326,3,77327.62
12347,2,8,4620.32
12348,75,5,1658.40
12349,19,3,3319.06
12350,310,1,258.00
...,...,...,...
18283,4,22,2527.45
18284,432,1,391.28
18285,661,1,361.10


In [23]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,5812.000000,5812.000000,5812.000000
mean,199.701136,6.199931,2641.060447
std,208.191682,12.519513,13167.396763
min,1.000000,1.000000,0.000000
25%,25.000000,1.000000,307.775000
50%,94.000000,3.000000,791.675000
75%,379.000000,7.000000,2058.422500
max,738.000000,372.000000,553408.560000


In [24]:
rfm['Country'] = df.groupby('Customer_ID')['Country'].agg(lambda x:x.value_counts().index[0])

rfm

,Recency,Frequency,MonetaryValue,Country
Customer_ID,,,,
12346,326,3,77327.62,United_Kingdom
12347,2,8,4620.32,Iceland
12348,75,5,1658.40,Finland
12349,19,3,3319.06,Italy
12350,310,1,258.00,Norway
...,...,...,...,...
18283,4,22,2527.45,United_Kingdom
18284,432,1,391.28,United_Kingdom
18285,661,1,361.10,United_Kingdom


In [25]:
# cherche l'index de la ligne avec le Customer_ID 12346
rfm.loc[12423]# Agréger les données par 'Customer_ID' et 'Invoice'
invoice_agg = df.groupby(['Customer_ID', 'Invoice']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Calcul de la récence par facture
    'Quantity': 'sum',  # Somme des quantités par facture
    'TotalSales': 'sum'  # Somme des ventes par facture
})

Recency                1
Frequency             10
MonetaryValue    2021.85
Country          Belgium
Name: 12423, dtype: object

In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer un sous-ensemble de données avec les 3 variables
rfm_rfm = rfm[['Recency', 'Frequency', 'MonetaryValue']]

# Créer une figure avec trois sous-graphiques
fig = make_subplots(rows=3, cols=1, subplot_titles=('Recency', 'Frequency', 'MonetaryValue'))

# Ajouter les graphiques de boîtes à chaque sous-graphique
fig.add_trace(go.Box(y=rfm_rfm['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(y=rfm_rfm['Frequency'], name='Frequency'), row=2, col=1)
fig.add_trace(go.Box(y=rfm_rfm['MonetaryValue'], name='MonetaryValue'), row=3, col=1)

# Mettre à jour le titre et les dimensions de la figure
fig.update_layout(title='Distribution des variables RFM',
                  height=800, width=600)

# Afficher la figure interactive dans le notebook
fig.show()


In [27]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,5812.000000,5812.000000,5812.000000
mean,199.701136,6.199931,2641.060447
std,208.191682,12.519513,13167.396763
min,1.000000,1.000000,0.000000
25%,25.000000,1.000000,307.775000
50%,94.000000,3.000000,791.675000
75%,379.000000,7.000000,2058.422500
max,738.000000,372.000000,553408.560000


In [28]:
# Define the upper limit for each variable
recency_upper_limit = rfm['Recency'].quantile(0.95)
frequency_upper_limit = rfm['Frequency'].quantile(0.95)
monetary_value_upper_limit = rfm['MonetaryValue'].quantile(0.95)

# Remove the outliers
rfm = rfm[(rfm['Recency'] <= recency_upper_limit) & 
          (rfm['Frequency'] <= frequency_upper_limit) & 
          (rfm['MonetaryValue'] <= monetary_value_upper_limit)]

In [29]:
rfm.describe()

,Recency,Frequency,MonetaryValue
count,5138.000000,5138.000000,5138.000000
mean,184.799728,4.352472,1315.068153
std,182.509279,3.980258,1488.993785
min,1.000000,1.000000,0.000000
25%,29.000000,1.000000,315.475000
50%,95.500000,3.000000,758.260000
75%,364.000000,6.000000,1706.202500
max,622.000000,20.000000,8438.340000


In [30]:
# Créer un sous-ensemble de données avec les 3 variables
rfm_rfm = rfm[['Recency', 'Frequency', 'MonetaryValue']]

# Créer une figure avec trois sous-graphiques
fig = make_subplots(rows=3, cols=1, subplot_titles=('Recency', 'Frequency', 'MonetaryValue'))

# Ajouter les graphiques de boîtes à chaque sous-graphique
fig.add_trace(go.Box(y=rfm_rfm['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(y=rfm_rfm['Frequency'], name='Frequency'), row=2, col=1)
fig.add_trace(go.Box(y=rfm_rfm['MonetaryValue'], name='MonetaryValue'), row=3, col=1)

# Mettre à jour le titre et les dimensions de la figure
fig.update_layout(title='Distribution des variables RFM',
                  height=800, width=600)

# Afficher la figure interactive dans le notebook
fig.show()

In [31]:
rfm.shape

(5138, 4)

In [32]:
# met le rfm dans un fichier csv
rfm.to_csv("../../data/df_rfm_model.csv" , index = 'Customer_ID')